In [1]:
import os
import numpy as np

import torch
import torch.nn as nn 
import torch.nn.functional as F 
from collections import OrderedDict
from PIL import Image
import torchvision.transforms as transforms

from models.iresnet import iresnet50
from models.evidential import edl_mse_loss, edl_digamma_loss, edl_log_loss, relu_evidence, exp_evidence, get_device, one_hot_embedding

import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm

/home/cry/anaconda3/envs/hsic/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# VGGFace2
num_classes = 8631
pre_trained = "ckpt/ArcFace-VGGFace2-R50-8631.pth"
device = "cuda:0"

In [2]:
# VGGFace2
num_classes = 10177
pre_trained = "ckpt/ArcFace-CelebA-R50-10177.pth"
device = "cuda:0"
img_path = "motivation/images/Celeb-A/ID/32/000492.jpg"

In [3]:
model = iresnet50(num_classes)
if pre_trained is not None and os.path.exists(pre_trained):
    # No related
    model_dict = model.state_dict()
    pretrained_param = torch.load(pre_trained)
    try:
        pretrained_param = pretrained_param.state_dict()
    except:
        pass

    new_state_dict = OrderedDict()
    for k, v in pretrained_param.items():
        if k in model_dict:
            new_state_dict[k] = v
            print("Load parameter {}".format(k))
        elif k[7:] in model_dict:
            new_state_dict[k[7:]] = v
            print("Load parameter {}".format(k[7:]))
        # else:
        #     print("Parameter {} unload.".format(k[7:]))

    model_dict.update(new_state_dict)
    model.load_state_dict(model_dict)
    print("Success load pre-trained model {}".format(pre_trained))
else:
    print("not load pretrained")

# Set to device
model.to(device)
model.eval()

Load parameter weight
Load parameter conv1.weight
Load parameter bn1.weight
Load parameter bn1.bias
Load parameter bn1.running_mean
Load parameter bn1.running_var
Load parameter bn1.num_batches_tracked
Load parameter prelu.weight
Load parameter layer1.0.bn1.weight
Load parameter layer1.0.bn1.bias
Load parameter layer1.0.bn1.running_mean
Load parameter layer1.0.bn1.running_var
Load parameter layer1.0.bn1.num_batches_tracked
Load parameter layer1.0.conv1.weight
Load parameter layer1.0.bn2.weight
Load parameter layer1.0.bn2.bias
Load parameter layer1.0.bn2.running_mean
Load parameter layer1.0.bn2.running_var
Load parameter layer1.0.bn2.num_batches_tracked
Load parameter layer1.0.prelu.weight
Load parameter layer1.0.conv2.weight
Load parameter layer1.0.bn3.weight
Load parameter layer1.0.bn3.bias
Load parameter layer1.0.bn3.running_mean
Load parameter layer1.0.bn3.running_var
Load parameter layer1.0.bn3.num_batches_tracked
Load parameter layer1.0.downsample.0.weight
Load parameter layer1.0.

IResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (prelu): PReLU(num_parameters=64)
  (layer1): Sequential(
    (0): IBasicBlock(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=64)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): IBas

In [4]:
transforms = transforms.Compose([
            transforms.Resize((112,112)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
        ])

In [5]:
img = Image.open(img_path)
img = transforms(img).unsqueeze(0)

In [6]:
model(img.to(device))

tensor([[ 0.0283,  0.0166,  0.0308,  ..., -0.0678, -0.0817,  0.0233]],
       device='cuda:0', grad_fn=<MmBackward0>)

In [7]:
model(img.to(device)).argmax()

tensor(32, device='cuda:0')

In [8]:
batch_size = 1  #批处理大小
input_shape = (3, 112, 112)   #输入数据,改成自己的输入shape

In [9]:
x = torch.randn(batch_size, *input_shape)   # 生成张量
x = x.to(device)
export_onnx_file = pre_trained.replace(".pth", ".onnx")		# 目的ONNX文件名
torch.onnx.export(model,
                    x,
                    export_onnx_file,
                    opset_version=10,
                    do_constant_folding=True,	# 是否执行常量折叠优化
                    input_names=["input"],	# 输入名
                    output_names=["output"],	# 输出名
                    dynamic_axes={"input":{0:"batch_size"},  # 批处理变量
                                    "output":{0:"batch_size"}})

## ONNX

In [1]:
import onnx_tf.backend
import onnx

import numpy as np
from PIL import Image
import torchvision.transforms as transforms

2023-04-28 17:07:01.283868: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-28 17:07:01.329293: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 17:07:02.021985: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/cry/anaconda3/envs/hsic/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode

In [2]:
transforms = transforms.Compose([
            transforms.Resize((112,112)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
        ])

In [3]:
img_path = "motivation/images/Celeb-A/ID/32/000492.jpg"
model_path = "ckpt/ArcFace-CelebA-R50-10177.onnx"

In [4]:
img = Image.open(img_path)
img = np.array([transforms(img).numpy()])

In [5]:
onnx_model = onnx.load(model_path)
tf_model = onnx_tf.backend.prepare(onnx_model, device='GPU:0')

2023-04-28 17:07:38.027694: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2027] Ignoring visible gpu device (device: 1, name: Quadro K620, pci bus id: 0000:17:00.0, compute capability: 5.0) with core count: 3. The minimum required count is 8. You can adjust this requirement with the env var TF_MIN_GPU_MULTIPROCESSOR_COUNT.
2023-04-28 17:07:38.672074: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21580 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:b3:00.0, compute capability: 8.6


In [6]:
result = tf_model.run(img)

2023-04-28 17:07:49.730604: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-04-28 17:07:50.896870: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [7]:
prob = np.argsort(result[0])[::-1]

In [8]:
result[0]

array([[ 0.02835666,  0.0165917 ,  0.03075849, ..., -0.06780577,
        -0.08168626,  0.02329685]], dtype=float32)